motive: We are attempting to use a constant-scaling method to perform endpoint estimations for beta and (possibly) gamma distributions. 
Basic steps are as follows:
1. Simulate a bunch of points and take some upper proportion p (upper thirds, upper quartile, etc.)
2. Calculate the standard deviation of the sample
3. Figure out what constant $c * \hat{\sigma}$ added to the sample maximum would get close to 1
4. Repeat 1-3 and obtain average of $c$ s.
5. Test on different Beta distributions (and maybe Gamma distributions). Is there a general rule?

In [1]:
import numpy as np

In [2]:
#Start with one instance of Beta(2,5) distribution
rng = np.random.default_rng(seed=2024)
sample = rng.beta(2,5,1000) #sample 1000 points from distribution
cutoff = np.percentile(sample, 66.7) #find cutoff for top third
test_sample = sample[sample > cutoff] #get top third
sd = np.std(test_sample) #get standard deviation
c = (1-np.max(test_sample)) / sd #get constant
print(c)

1.4094303685479377


In [3]:
#Repeating steps 1-3 on the same Beta(2,5) distribution
for seed in np.array([2,34,567,2021,0]):
    rng = np.random.default_rng(seed=seed)
    sample = rng.beta(2,5,1000) #sample 1000 points from distribution
    cutoff = np.percentile(sample, 66.7) #find cutoff for top third
    test_sample = sample[sample > cutoff] #get top third
    sd = np.std(test_sample) #get standard deviation
    c = (1-np.max(test_sample)) / sd #get constant
    

In [4]:
cs = []
for seed in np.array([2,34,567,2021,0]):
    rng = np.random.default_rng(seed=seed)
    sample = rng.beta(2,5,1000) 
    cutoff = np.percentile(sample, 66.7) 
    test_sample = sample[sample > cutoff] 
    sd = np.std(test_sample) 
    c = (1-np.max(test_sample)) / sd 
    cs.append(c)
print(np.average(cs))

1.476260546408297


In [5]:
avg_cs = []
for sample_sizes in np.array([200,500,1000,5000]):
    cs = []
    for seed in np.array([2,34,567,2021,0]):
        rng = np.random.default_rng(seed=seed)
        sample = rng.beta(2,5,sample_sizes) 
        cutoff = np.percentile(sample, 66.7) 
        test_sample = sample[sample > cutoff] 
        sd = np.std(test_sample) 
        c = (1-np.max(test_sample)) / sd 
        cs.append(c)
    avg_cs.append(np.average(cs))
print(avg_cs)

[2.5413629722180695, 1.8449258635121517, 1.476260546408297, 1.2829521053560058]


In [6]:
parameter_cs = []
for b in np.linspace(0.5,5,10):
    sample_avg_cs = []
    for sample_sizes in np.array([200,500,1000,5000]):
        cs = []
        for seed in np.array([2,34,567,2021,0]):
            rng = np.random.default_rng(seed=seed)
            sample = rng.beta(2,b,sample_sizes) 
            cutoff = np.percentile(sample, 66.7) 
            test_sample = sample[sample > cutoff] 
            sd = np.std(test_sample) 
            c = (1-np.max(test_sample)) / sd 
            cs.append(c)
        sample_avg_cs.append(np.average(cs))
    parameter_cs.append(np.average(sample_avg_cs))
print(parameter_cs)

[0.0007143143689919895, 0.01182623376629679, 0.12740538018372408, 0.24249472650593012, 0.4345173669513564, 0.6256447408936548, 0.8664725621306946, 1.149091351673057, 1.4463983562538267, 1.786375371873631]


In [7]:
#I am paranoid now so I will generate different values of a and b first
parameter_pairs = []
for a in np.linspace(0.5,10):
    for b in np.linspace(0.5,10):
        parameter_pairs.append((a,b))

In [8]:
for pair in parameter_pairs:
    sample_avg_cs = []
    for sample_sizes in np.array([200,500,1000,5000]):
        cs = []
        for seed in np.array([2,34,567,2021,0]):
            rng = np.random.default_rng(seed=seed)
            sample = rng.beta(pair[0],pair[1],sample_sizes) 
            test_sample = sample[sample >= np.percentile(sample, 66.7)]
            c = (1-np.max(test_sample)) / np.std(test_sample)
            cs.append(c)
        sample_avg_cs.append(np.average(cs))
    parameter_cs.append(np.average(sample_avg_cs))

print(parameter_cs)

[0.0007143143689919895, 0.01182623376629679, 0.12740538018372408, 0.24249472650593012, 0.4345173669513564, 0.6256447408936548, 0.8664725621306946, 1.149091351673057, 1.4463983562538267, 1.786375371873631, 0.0005123196800882045, 0.004353955479425375, 0.013170698405716684, 0.040830755132690584, 0.08570136739198997, 0.1304149054738965, 0.1885725055639946, 0.2782720476167248, 0.30002254741800105, 0.32606329739339746, 0.5186519590678849, 0.6419272338272672, 0.677945580979993, 0.8261715861828318, 0.9739410507955609, 1.1435751975915163, 1.3241310197732201, 1.5130568728679274, 1.676407959865594, 1.8698607521867041, 2.0678302587701993, 2.267489933758548, 2.467876882976026, 2.6742056597506636, 2.884654498444537, 3.0931778852338274, 3.2076443250142246, 3.4156386864028985, 3.626832916972866, 3.8150530744352364, 4.020529417310995, 4.23053421744812, 4.143186986016702, 4.348922554310495, 4.555002926345833, 4.766336319085196, 4.976557453354661, 5.18741299415438, 5.401088084442556, 5.616258037332637, 5

In [9]:
import plotly.graph_objects as go

def graph_objects(params, cs):
    x, y = zip(*params)
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=cs,
                                       mode='markers', 
                                       marker=dict(
                                       size=12,
                                       color=cs,                # set color to an array/list of desired values
                                       colorscale='Viridis',   # choose a colorscale
                                       opacity=0.8
        ))])
    fig.update_layout(scene = dict(
                        xaxis_title='alpha',
                        yaxis_title='beta',
                        zaxis_title='scaling constant'),
                        margin=dict(r=20, b=10, l=10, t=10))
    fig.show()

In [10]:
graph_objects(parameter_pairs, parameter_cs)